#Sequential Model Examples with Keras

## Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

In [ ]:
# Set random seed for reproducibility
np.random.seed(40)
import tensorflow as tf
tf.random.set_seed(40)

## Data Processing

In [ ]:
# Load the dataset
data_url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv"
df = pd.read_csv(data_url)

# Extract and reshape the temperature values
temperatures = (
    df.reset_index(drop=True)["Temp"].values.astype(float).reshape(-1, 1))

# Scale the temperature data to a normalized range [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1))
data_normalized = scaler.fit_transform(temperatures)

In [ ]:
def create_sequences(data, seq_length):
    """ Generates input-output pairs from the provided data."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 20
X, y = create_sequences(data_normalized, seq_length)

##Training and evaluating

In [ ]:
# Split the dataset into training and testing sets with a 67% training size
train_size = int(len(X) * 0.67)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
# Model definition using Keras
input_size = 1
hidden_size = 50
output_size = 1
num_layers = 2
learning_rate = 0.001
num_epochs = 100
l2_penalty = 0.01

model = Sequential()
model.add(LSTM(hidden_size, return_sequences=False, input_shape=(seq_length, input_size),
               kernel_regularizer=l2(l2_penalty)))
model.add(Dense(output_size))

In [ ]:
# Instantiate LSTM model, define loss function, and optimizer
model.compile(optimizer=Adam(learning_rate=learning_rate), loss="mse")


In [ ]:
# Train model
model.fit(X_train, y_train, epochs=num_epochs, batch_size=32, verbose=1)


Epoch 1/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 0.0474
Epoch 2/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0178
Epoch 3/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0145
Epoch 4/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 0.0133
Epoch 5/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 0.0126
Epoch 6/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 0.0122
Epoch 7/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0119
Epoch 8/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0117
Epoch 9/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0115
Epoch 10/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0114
Epoch 11/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0113
Epoch 12/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0113
Epoch 13/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0112
Epoch 14/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.0112
Epoch 15/100
76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 0.011

In [ ]:
# Predicting with the model using test data and inverting the scaling of predictions
y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
print("Predicted values:", y_pred_inv.squeeze())

# Calculate Mean Squared Error between true and predicted values
mse = mean_squared_error(scaler.inverse_transform(y_test), y_pred_inv.squeeze())
print("Mean Squared Error:", mse)

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
Predicted values: [11.58172   7.503418  7.181761 ... 13.409643 13.399893 14.637098]
Mean Squared Error: 5.416724421509333
